In [1]:
%pip install audiomentations



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from audiomentations import Compose, Gain, GainTransition, AddGaussianNoise,AddBackgroundNoise, TimeStretch, PitchShift, Shift
import numpy as np
import os 
import keras.layers as layers
import tensorflow as tf
import tensorflow_io as tfio
import librosa
import librosa.display


In [ ]:


augment = Compose([
    AddBackgroundNoise(sounds_path="./background_noises"), 
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    Gain(),
    Shift(min_shift=-0.05, max_shift=0.05, p=0.5),
])


class Augmentation(layers.Layer):
  def __init__(self, augmentation, sample_rate=32000):
    super(Augmentation, self).__init__()
    self.augmentation = augmentation
    self.sample_rate = sample_rate
        
  def call(self, waves):
    waves = self.augmentation(samples=waves, sample_rate=self.sample_rate)
    return waves

In [ ]:
augmentLayer = Augmentation(augment, 16000)




Rename file in folder to format like 00011.wav

In [ ]:
folder_lim_path = 'raw_data_1s'
batch_size = 32
seed = 42
desired_sr=44100

# Get list of file paths and corresponding labels
file_paths = []
labels = []
def load_data( folder_lim_path, folder_name):
# Rename file
    files = []
    labels = []
    path = os.path.join( folder_lim_path, folder_name)
    for f in os.listdir(path):
        full_name = os.path.join(path, f)
        files.append(full_name)
        labels.append(folder_name)

    
    return files, labels


In [ ]:
file_paths =[]
labels=[]
f,l = load_data( folder_lim_path, 'pushka')
file_paths+=f
labels+=l
f,l = load_data( folder_lim_path, 'none')
file_paths+=f
labels+=l
f,l = load_data( folder_lim_path, 'geran')
file_paths+=f
labels+=l

# Shuffle data and split into training and validation sets
num_samples = len(file_paths)
indices = np.random.permutation(num_samples)
split = int(0.8 * num_samples)  # 80-20 split

train_paths = [file_paths[i] for i in indices[:split]]
train_labels = [labels[i] for i in indices[:split]]

val_paths = [file_paths[i] for i in indices[split:]]
val_labels = [labels[i] for i in indices[split:]]

# Create datasets for training and validation
print(train_paths)
print(train_labels)
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

# Function to load and process audio files


# Shuffle data and split into training and validation sets
num_samples = len(file_paths)
indices = np.random.permutation(num_samples)
split = int(0.8 * num_samples)  # 80-20 split

train_paths = [file_paths[i] for i in indices[:split]]
train_labels = [labels[i] for i in indices[:split]]

val_paths = [file_paths[i] for i in indices[split:]]
val_labels = [labels[i] for i in indices[split:]]


# Create datasets for training and validation
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

# Utility functions for loading audio files and making sure the sample rate is correct.

@tf.function
def load_wav_mono(filename, sr=32000):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=sr)
    return wav

def load_wav_for_map(filename, label):
  return load_wav_mono(filename, desired_sr), label

def load_and_augment_audio(file_path, label):
    audio = load_wav_mono(file_path, desired_sr)
    # audio = augment(samples=audio, sample_rate=desired_sr)
    return audio, label

train_dataset =     train_dataset.map(load_and_augment_audio, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.map(load_wav_for_map)
# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths), seed=seed)
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)



In [ ]:
def augmetion_audio_file_split(file_path, folder_lim_path, folder_name ,folder_lim_time, offset=1,  desired_sr=44100):
    # Создайте пустой DataFrame для хранения результатов

    folder  = os.path.join(folder_lim_path, folder_name)
    if not os.path.exists(folder):
        os.makedirs(folder)
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    folder  = os.path.join(folder_path, folder_name)
    for filename in os.listdir(folder):
        if filename.endswith('.wav'): 
            file_path = os.path.join(folder_path, folder_name, filename)
            audio, sample_rate = librosa.load(file_path, sr=desired_sr)
            # Получение длины в секундах  
            audio_length = len(audio) / sample_rate
            folder_lim_time_sample = int(folder_lim_time*sample_rate)
            offset_sample = int(offset*folder_lim_time_sample)
            start_sample=0
            end_sample=folder_lim_time_sample
            sub_number =0
            while end_sample <= len(audio):
                sub_number = sub_number+1
                audio_split = audio[start_sample:end_sample]
                filename_new = filename[:-4] + f"_{sub_number:03}.wav"
                file_path_lim = os.path.join(folder_lim_path,folder_name, filename_new)
                sf.write(file_path_lim, audio_split, sample_rate, subtype='PCM_24')

                start_sample=start_sample+offset_sample
                end_sample=end_sample+offset_sample
                



def augmetion_audio_file_select(file_path, folder_lim_path, folder_name ,folder_lim_time, event_start=0.3, desired_sr=44100):
    
    # Specify the desired sampling rate (e.g., 44100 Hz)
    # desired_sr 

    # Создайте пустой DataFrame для хранения результатов
    file = []
    pick = []
    power = []

    # Путь к папке, содержащей ваши WAV-файлы
    data_csv = {'File':[], 'Pick':[], 'Power':[]}
    ##
    folder  = os.path.join(folder_lim_path, folder_name)
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    folder  = os.path.join(folder_path, folder_name)
    for filename in os.listdir(folder):
        if filename.endswith('.wav'): 
            file_path = os.path.join(folder_path, folder_name, filename)
            audio, sample_rate = librosa.load(file_path, sr=desired_sr)
            # Получение длины в секундах  
            audio_length = len(audio) / sample_rate
            ( jump_x, jump_y  ) = get_auduo_jump_power(audio, sample_rate)
            if(len(jump_x)==0):
                print(filename, jump_x)
            else:
                data_csv['File'].append(filename)
                data_csv['Pick'].append(jump_x[0])
                data_csv ['Power'].append(jump_y[0])
                folder_lim_time_sample = int(folder_lim_time*sample_rate)
                if audio_length < 1:
                    padding = np.zeros(int(folder_lim_time_sample - len(audio)))
                    audio = np.concatenate(( padding, audio))
                else:
                    pik_time = jump_x[0]
                    pik_time_sample = int(pik_time*sample_rate)
                    start_time = pik_time_sample-int(folder_lim_time_sample*event_start)
                    pik_times=[start_time, start_time+folder_lim_time_sample]
                    if pik_times[0] < 0:
                        pik_times=[0, folder_lim_time_sample]
                    if pik_times[1] > len(audio):
                        pik_times=[len(audio-folder_lim_time_sample), len(audio)]

                    audio = audio[pik_times[0]:pik_times[1]]
                file_path_lim = os.path.join(folder_lim_path,folder_name, filename)
                sf.write(file_path_lim, audio, sample_rate, subtype='PCM_24')

    df = pd.DataFrame(data_csv) 

    # сохранение в Csv
    df.to_csv(os.path.join(folder_path, f'{folder_name}.xlsx'))


In [ ]:
folder_path = 'raw_data'
folder_lim_path = 'raw_data_1s'
folder_name = 'pushka'
folder_lim_time = 1
augmetion_audio_file_select(folder_path, folder_lim_path, folder_name, folder_lim_time)

In [ ]:
folder_path = 'raw_data'
folder_lim_path = 'raw_data_1s'
folder_lim_time = 1
augmetion_audio_file_split(folder_path, folder_lim_path, 'geran', folder_lim_time)
augmetion_audio_file_split(folder_path, folder_lim_path, 'none', folder_lim_time)
augmetion_audio_file_split(folder_path, folder_lim_path, 'roked', folder_lim_time)
augmetion_audio_file_split(folder_path, folder_lim_path, 'shots', folder_lim_time)


In [ ]:

file_name = "00316.wav"
file_path = os.path.join(folder_path,folder_name, file_name)
ipd.Audio(file_path)
file_path = os.path.join(folder_lim_path,folder_name, file_name)
ipd.Audio(file_path)


In [ ]:
audio, sample_rate = librosa.load(file_path, sr=desired_sr)


In [ ]:

# Need show wave from audio array       

plt.subplots()
librosa.display.waveshow(audio, sr=sample_rate)
plt.title(file_name)


In [ ]:
frameSize = 2048
hopSize = 512
stft_audio = librosa.stft(audio, n_fft = frameSize, hop_length = hopSize)
D_harmonic, D_percussive = librosa.decompose.hpss(stft_audio)

stft_audio_db = librosa.amplitude_to_db(np.abs(stft_audio), ref=np.max)
print(stft_audio.shape)

In [ ]:


get_auduo_jump_power(audio, sample_rate, show_chart=True)


In [ ]:
stft_audio = librosa.stft(audio)#, n_fft = frameSize, hop_length = hopSize)
D_harmonic, D_percussive = librosa.decompose.hpss(stft_audio)

#rp = np.max(np.abs(stft_audio))
rp = np.max(stft_audio )
print(rp)
rp = np.max

plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
librosa.display.specshow(librosa.power_to_db(stft_audio, ref=rp),sr=sample_rate, y_axis='log')
plt.colorbar()
plt.title('Full spectrogram')

plt.subplot(3, 1, 2)
librosa.display.specshow(librosa.power_to_db(D_harmonic, ref=rp), sr=sample_rate,y_axis='log')
plt.colorbar()
plt.title('Harmonic spectrogram')

plt.subplot(3, 1, 3)
librosa.display.specshow(librosa.power_to_db(D_percussive, ref=rp),sr=sample_rate, y_axis='log', x_axis='time')
plt.colorbar()
plt.title('Percussive spectrogram')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
img = librosa.display.specshow(stft_audio_db,sr=sample_rate, x_axis='time', y_axis='log', ax=ax)
ax.set(title='Now with labeled axes!')
fig.colorbar(img, ax=ax, format="%+2.f dB")


In [ ]:
fig, ax = plt.subplots()
M = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
M_db = librosa.power_to_db(M, ref=np.max)
M_db = librosa.amplitude_to_db(M, ref=np.max)
img = librosa.display.specshow(M_db, y_axis='mel',sr=sample_rate, x_axis='time', ax=ax)
ax.set(title='Mel spectrogram display')
fig.colorbar(img, ax=ax, format="%+2.f dB")